In [2]:
# We're doing some basic deduplication removing exact matches for street number 
# + strong fuzzy matches for street names. 

In [1]:
import pandas as pd

from thefuzz import fuzz
from thefuzz import process

/home/jovyan/.local/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [18]:
df = pd.read_csv('formatted_25.csv', low_memory=False)
print('rows: ', len(df))

rows:  803658


In [7]:
type(df)

pandas.core.frame.DataFrame

In [20]:
# Start by removing exact matches

# df = df.drop_duplicates(subset=['street_no','formatted_en', 'province'])
# df = df[(~df.duplicated(subset=['street_no','formatted_en', 'province'])) | df['street_no','formatted_en', 'province'].isna()]

df = df[~df.duplicated(subset=['street_no','formatted_en', 'province']) | df['street_no'].isna() | df['formatted_en'].isna() | df['province'].isna()]

print('rows: ', len(df))
# if (province_code == 'QC'):
#     df = df.drop_duplicates(subset=['street_no','formatted_fr'])
# else:
#     df = df.drop_duplicates(subset=['street_no','formatted_en'])

rows:  676262


In [8]:
# ok, so it looks like a lot of results might be duplicated quite simply
# now, let's find out how many we have to geocode. and how many simple 
# deduplication removes
list(df)

['localfile',
 'province',
 'mailing_address',
 'street_no',
 'street_name',
 'latitude',
 'longitude',
 'formatted_en',
 'formatted_fr']

In [15]:
df

,localfile,province,mailing_address,street_no,street_name,latitude,longitude,formatted_en,formatted_fr
0,BC_Victoria_Business_Licences.csv,BC,NaN,1417,PEMBROKE ST,48.431011,-123.341208,pembroke st,PEMBROKE ST
7108,AB_Banff_Business_Licences.csv,AB,NaN,202,BANFF AVE,NaN,NaN,banff ave,BANFF AVE
8050,NB_Saint_John_Grocery_Stores.csv,NB,NaN,NaN,NaN,45.276229,-66.054280,NaN,NaN
41744,ON_Hamilton_Licenced_Public_Halls.csv,ON,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62083,ON_Pickering_Business_Directory.csv,NaN,NaN,1266,Townline,44.105207,-78.944945,tline,Townline
72035,BC_Vancouver_Business_Licences.csv,QC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78598,BC_Vancouver_Business_Licences.csv,MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85799,BC_Vancouver_Business_Licences.csv,SK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89630,BC_Vancouver_Business_Licences.csv,NS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101201,BC_Vancouver_Business_Licences.csv,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# how many to geocode?
print(len(df[df['longitude'].isnull()]))

35824


In [26]:
# ok, 'only' 35824. Is that right?? That's not too many at all. And it'll be way less with matching. 
# do I need to match any addresses that are geocoded already? or is it just for geocoding? prob former

In [30]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]
})

In [13]:
df['province'].unique()

array(['BC', 'AB', 'NB', 'ON', nan, 'QC', 'MB', 'SK', 'NS', '78', 'YT',
       'NL', 'LAKEVIEW ESTATES', 'NT', 'NU', 'PE'], dtype=object)

In [32]:
df.drop_duplicates(subset=['brand','style'])

,brand,style,rating
0,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
